In [1]:
import pygame as pg
from random import sample

pygame 2.0.0 (SDL 2.0.12, python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
switch={
    #pg.K_RETURN:'RETURN',
    pg.K_ESCAPE:'ESCAPE',       #Exit the program
    #pg.K_t:'T',
    #pg.K_c:'C',
    #pg.K_UP:'UP',               #Used for cell selection
    #pg.K_DOWN:'DOWN',           #Used for cell selection
    #pg.K_LEFT:'LEFT',           #Used for cell selection
    #pg.K_RIGHT:'RIGHT',         #Used for cell selection
    #pg.K_o:'O',
    #pg.K_p:'P',
    #pg.K_l:'L',
    #pg.K_k:'K',
    #pg.K_j:'J',
    'MOUSE_PRESS':'MOUSE_PRESS' #Used to select a cell and indicate that an action should be performed
}       
        
def keypress(key):
    """ Given a pressed key, check if we care whether this key has been pressed. """
    return switch.get(key, "SP_NONE")

def getKey():
    """ Looks for the states of a set of keys and whether the QUIT key has been pressed. """
    key = None
    for event in pg.event.get():                   #For all pygame events
        if event.type == pg.QUIT:                  #If the user is trying to quit
            key = pg.K_ESCAPE                      #Handle as an escape key
        elif event.type == pg.KEYDOWN:             #If the user is pressing a key
            key = event.key                        #Get the key
        elif event.type == pg.MOUSEBUTTONDOWN:     #If the user has used a mouse button
            mouse_presses = pg.mouse.get_pressed() #Get what they've pressed
            if mouse_presses[0]:                   #If they've pressed left mouse
                key = 'MOUSE_PRESS'                #Set the key as a mousepress
    return keypress(key)                           #Return the result of the switch statement

In [3]:
class CellState:
    def __init__(self, word="NOT SET", word_type="N"):
        self.word = word
        self.word_type = word_type
        self.known = False

In [4]:
class UserInterface:
    def __init__(self):
        #pg.display.init()
        self.cell_r_img = pg.image.load("img/cellr.png")
        self.cell_b_img = pg.image.load("img/cellb.png")
        self.cell_x_img = pg.image.load("img/cellx.png")
        self.cell_n_img = pg.image.load("img/celln.png")
        self.bg_img   = pg.image.load("img/bg.png")
        self.cell_size = (256, 128)
        
        self.font = pg.font.SysFont("calibri", 36)
        
        self.cells_x = 5
        self.cells_y = 6
        
        self.screen = pg.display.set_mode((self.cells_x*self.cell_size[0], self.cells_y*self.cell_size[1]))
    def update_screen(self, cells_states):
        self.update_background()
        self.update_cells(cells_states, self.cell_size[1]*1)
        
        pg.display.update()
    def update_cells(self, cells_states, y_offset=0):
        r_img = pg.transform.scale(self.cell_r_img, self.cell_size)
        b_img = pg.transform.scale(self.cell_b_img, self.cell_size)
        x_img = pg.transform.scale(self.cell_x_img, self.cell_size)
        n_img = pg.transform.scale(self.cell_n_img, self.cell_size)
        for x in range(0, 5):
            for y in range(0, 5):
                cell_states = cells_states[x][y]
                pos = (self.cell_size[0]*x, self.cell_size[1]*y + y_offset)
                word_type = cell_states.word_type
                known = cell_states.known
                
                #Display cell card
                if word_type == "R" and known:
                    self.screen.blit(r_img, pos)
                elif word_type == "B" and known:
                    self.screen.blit(b_img, pos)
                elif word_type == "X" and known:
                    self.screen.blit(x_img, pos)
                else:
                    self.screen.blit(n_img, pos)
                
                #Display cell word
                word = self.font.render(cell_states.word, 1, (0,0,0))                
                self.screen.blit(word, (pos[0] + int(self.cell_size[0]/3),
                                        pos[1] + int(self.cell_size[1]/3)))
    def update_background(self):
        bg_img = pg.transform.scale(self.bg_img, self.cell_size)
        for x in range(0, self.cells_x):
            for y in range(0, self.cells_y):
                pos = (self.cell_size[0]*x, self.cell_size[1]*y)
                #Display cell background
                self.screen.blit(bg_img, pos)
                
                
    def __del__(self):
        pg.display.quit()
        

In [5]:
def get_wordlist():
    wordlist = []
    with open("wordlist.txt", 'r') as wordfile:
        lines = wordfile.readlines()
        for line in lines:
            wordlist.append(line[0:-1])
    return wordlist

In [6]:
def main(UI):
    #Get wordlist
    wordlist = get_wordlist()
    
    #Generate word cards by randomly selecting words from the list, then choose their state
    selected_words = sample(wordlist, k=25)
    assigned_words = sample(selected_words, k=25)
    red_words = assigned_words[0:9]
    blue_words = assigned_words[9:17]
    fail_words = [assigned_words[17]]    
    
    cell_states = []
    
    #Add all cell information
    for i in range(0, 5):
        cell_state_arr = []
        for j in range(0, 5):
            index_1D = i + j*5
            
            #Set the cell state (Red, Blue, Fail, or Neutral)
            if selected_words[index_1D] in red_words:
                word_type = "R"
            elif selected_words[index_1D] in blue_words:
                word_type = "B"
            elif selected_words[index_1D] in fail_words:
                word_type = "X"
            else:
                word_type = "N"
            cell_state = CellState(selected_words[index_1D], word_type)
            cell_state_arr.append(cell_state)
        cell_states.append(cell_state_arr)
    
    #Set the user's selected key as none
    key = "SP_INITIAL"
    mouse = pg.mouse.get_pos() #Get the current location of the mouse 

    #While the user isn't trying to quit
    done = False
    while not done:
        UI.update_screen(cell_states)
        
        if key in ("ESCAPE"):
            done = True
        elif key in ("MOUSE_PRESS"):
            #For now, if the user clicks the screen then display all locations
            for i in range(0, len(cell_states)):
                for j in range(0, len(cell_states[i])):
                    cell_states[i][j].known = True
            
            
        key = getKey() #Get any relevant keys being pressed
        
        mouse = pg.mouse.get_pos() #Get the current location of the mouse 
    
    

try:
    pg.init()
    UI = UserInterface()
    main(UI)
finally:
    del UI
    pg.quit()